In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import undetected_chromedriver as uc
from io import StringIO

In [2]:
# Ler o df dos deputados
df_dep = pd.read_csv('./Info deputados.csv')

# Gastos Deputados Cota

Baixei os dados pelo site: https://dadosabertos.camara.leg.br/swagger/api.html?tab=staticfile#staticfile e vou filtrar para os deputados que quero analisar <br/> após isso vou precisar fazer um join para identificar os id do deputados. 

In [ ]:
import pandas as pd

df_dep_gasto = pd.read_csv('./csv/Gastos Cota Parlamentar/Gasto_dep_2025.csv', encoding='utf-8', sep=';')
df_dep_gasto.set_index('nuDeputadoId', inplace=True) # Deputado como index do df

df_deputados = pd.read_csv('./csv/Gastos Cota Parlamentar/deputados.csv', encoding='utf-8', sep=';')
df_deputados['uri'][0]





'https://dadosabertos.camara.leg.br/api/v2/deputados/220593'

*Os dados vieram IMCOMPLETOS é necessário usar a API para pegar os dados certos *

## API para deputados

### Teste

In [ ]:
import pandas as pd
import requests

# URL inicial (a que você passou)
url = "https://dadosabertos.camara.leg.br/api/v2/deputados/204362/despesas?ano=2025&mes=02&ordem=ASC&ordenarPor=ano"

todos_dados = [] # Lista vazia para acumular os resultados de todas as páginas

print("Iniciando coleta...")

while url:
    # 1. Faz a requisição
    response = requests.get(url)
    
    # Verifica se deu certo (código 200)
    if response.status_code == 200:
        conteudo = response.json()
        
        # 2. Adiciona os itens da página atual na nossa lista principal
        todos_dados.extend(conteudo['dados'])
        
        # 3. Descobre qual é a próxima página
        url = None # Reseta a url para garantir que o loop pare se não achar 'next'
        
        for link in conteudo['links']:
            if link['rel'] == 'next':
                url = link['href'] # Atualiza a url para a próxima volta do loop
                print(f"Indo para a próxima página: {url}")
                break
    else:
        print(f"Erro na requisição: {response.status_code}")
        break

# 4. Transforma a lista completa em DataFrame
df = pd.DataFrame(todos_dados)

print("Coleta finalizada!")
# Mostra o resultado final
df

Iniciando coleta...
Indo para a próxima página: https://dadosabertos.camara.leg.br/api/v2/deputados/204362/despesas?ano=2025&mes=02&ordem=ASC&ordenarPor=ano&pagina=2&itens=15
Indo para a próxima página: https://dadosabertos.camara.leg.br/api/v2/deputados/204362/despesas?ano=2025&mes=02&ordem=ASC&ordenarPor=ano&pagina=3&itens=15
Indo para a próxima página: https://dadosabertos.camara.leg.br/api/v2/deputados/204362/despesas?ano=2025&mes=02&ordem=ASC&ordenarPor=ano&pagina=4&itens=15
Indo para a próxima página: https://dadosabertos.camara.leg.br/api/v2/deputados/204362/despesas?ano=2025&mes=02&ordem=ASC&ordenarPor=ano&pagina=5&itens=15
Indo para a próxima página: https://dadosabertos.camara.leg.br/api/v2/deputados/204362/despesas?ano=2025&mes=02&ordem=ASC&ordenarPor=ano&pagina=6&itens=15
Indo para a próxima página: https://dadosabertos.camara.leg.br/api/v2/deputados/204362/despesas?ano=2025&mes=02&ordem=ASC&ordenarPor=ano&pagina=7&itens=15
Coleta finalizada!


,ano,mes,tipoDespesa,codDocumento,tipoDocumento,codTipoDocumento,dataDocumento,numDocumento,valorDocumento,urlDocumento,nomeFornecedor,cnpjCpfFornecedor,valorLiquido,valorGlosa,numRessarcimento,codLote,parcela
0,2025,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7870854,Nota Fiscal,0,2025-02-11T00:00:00,034670673,179.67,https://www.camara.leg.br/cota-parlamentar/doc...,CELESC DISTRIBUICAO S.A,08336783000190,174.00,5.67,,2111490,0
1,2025,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7870859,Nota Fiscal,0,2025-02-04T00:00:00,116745102502,714.84,https://www.camara.leg.br/cota-parlamentar/doc...,COMPANHIA CATARINENSE DE AGUAS E SANEAMENTO CASAN,82508433000117,452.06,262.78,,2111490,0
2,2025,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7884934,Nota Fiscal,0,2025-03-12T00:00:00,0019754129,311.85,https://www.camara.leg.br/cota-parlamentar/doc...,NET Claro NXT,66970229003930,311.85,0.00,,2119675,0
3,2025,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7873614,Recibos/Outros,1,2025-02-18T00:00:00,24880048,4529.29,https://www.camara.leg.br/cota-parlamentar/doc...,PREFEITURA MUNICIPAL DE CRICIÚMA,82916818000113,4529.29,0.00,,2112952,0
4,2025,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7864249,Nota Fiscal Eletrônica,4,2025-02-05T00:00:00,6286,286.50,http://www.camara.leg.br/cota-parlamentar/nota...,WMS COMERCIO DE ARTIGOS DE PAPELARIA LTDA,12132854000100,286.50,0.00,,2107692,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2025,2,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",7873459,Nota Fiscal,0,2025-02-04T00:00:00,609651,50.00,https://www.camara.leg.br/cota-parlamentar/doc...,VELOE COMPANINHA BRASILEIRA DE SOLUÇOES E SERV...,04740876000125,50.00,0.00,,2112952,0
89,2025,2,PASSAGEM AÉREA - SIGEPA,309688,Nota Fiscal,0,2025-02-16T12:00:00,1272132427280,1995.29,None,GOL,,1995.29,0.00,0,0,0
90,2025,2,PASSAGEM AÉREA - SIGEPA,308293,Nota Fiscal,0,2025-02-11T12:00:00,9572219431466,1327.92,None,TAM,,1327.92,0.00,0,0,0
91,2025,2,PASSAGEM AÉREA - SIGEPA,308748,Nota Fiscal,0,2025-02-12T12:00:00,9572219624681,2760.04,None,TAM,,2760.04,0.00,0,0,0


In [ ]:
df[(df['tipoDespesa'] == 'PASSAGEM AÉREA - SIGEPA')]['valorDocumento'].sum()

np.float64(7698.68)

### List deputados

In [ ]:
import pandas as pd
import requests

In [14]:
import pandas as pd
import requests

url = 'https://dadosabertos.camara.leg.br/api/v2/deputados?siglaUf=SC&ordem=ASC&ordenarPor=nome' # API oficial

response = requests.get(url) # Acessa API

if response.status_code == 200: # Se o resultado for sucesso
    df_deputados = pd.DataFrame(response.json()['dados']) # Transforma o conteudo em DataFrame
print(f"Encontrados {len(df_deputados)} deputados em SC.")

Encontrados 16 deputados em SC.


In [ ]:
dados = {
    'id': ['204361'], 
    'nome': ['CARLOS ALBERTO CHIODINI']
}

df_deputados = pd.DataFrame(dados)

In [ ]:
df_deputados

,id,nome
0,204361,CARLOS ALBERTO CHIODINI


In [16]:
lista_todas_despesas = []

for index, linha, in df_dep.iterrows():
    id_dep = linha['id']
    nome_dep = linha['nome']
    if nome_dep == 'Coronel Armando':
        print(f'Coletando dados de: {nome_dep} com ID -> ({id_dep})')
        
        for mes in range(1,13): # Pegar os 12 meses
            url_despesa = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{id_dep}/despesas?ano=2025&mes={mes}&ordem=ASC&ordenarPor=ano"
            
            # Loop de paginação (caso haja)
            while url_despesa:
                resp = requests.get(url_despesa)
                print(f'{url_despesa} \n')
                if resp.status_code == 200:
                    dados_json = resp.json()
                    despesas_pagina = dados_json['dados']
                    
                    # Se a lista não estiver vazia tem gastos no mes
                    if despesas_pagina:
                        for despesa in despesas_pagina:
                            despesa['id_deputado'] = id_dep
                            despesa['nome_deputado'] = nome_dep
                            
                        # Salvar na lista
                        lista_todas_despesas.extend(despesas_pagina)
                        
                    url_despesa = None
                    
                    for link in dados_json['links']:
                        if link['rel'] == 'next':
                            url_despesa = link['href']
                else:
                    print(f'Erro no mês {mes} de {nome_dep}')
                    break
        break

df_gastos_deputados = pd.DataFrame(lista_todas_despesas)
print(f"Coleta finalizada! Total de registros: {len(df_gastos_deputados)}")

Coletando dados de: Coronel Armando com ID -> (204376)
https://dadosabertos.camara.leg.br/api/v2/deputados/204376/despesas?ano=2025&mes=1&ordem=ASC&ordenarPor=ano 

https://dadosabertos.camara.leg.br/api/v2/deputados/204376/despesas?ano=2025&mes=2&ordem=ASC&ordenarPor=ano 

https://dadosabertos.camara.leg.br/api/v2/deputados/204376/despesas?ano=2025&mes=3&ordem=ASC&ordenarPor=ano 

https://dadosabertos.camara.leg.br/api/v2/deputados/204376/despesas?ano=2025&mes=4&ordem=ASC&ordenarPor=ano 

https://dadosabertos.camara.leg.br/api/v2/deputados/204376/despesas?ano=2025&mes=5&ordem=ASC&ordenarPor=ano 

https://dadosabertos.camara.leg.br/api/v2/deputados/204376/despesas?ano=2025&mes=5&ordem=ASC&ordenarPor=ano&pagina=2&itens=15 

https://dadosabertos.camara.leg.br/api/v2/deputados/204376/despesas?ano=2025&mes=5&ordem=ASC&ordenarPor=ano&pagina=3&itens=15 

https://dadosabertos.camara.leg.br/api/v2/deputados/204376/despesas?ano=2025&mes=5&ordem=ASC&ordenarPor=ano&pagina=4&itens=15 

https://dad

In [19]:
import pandas as pd
import requests
import time

# Listas para armazenar os dados e os erros
lista_todas_despesas = []
relatorio_erros = []

print(f"Iniciando coleta de {len(df_dep)} deputados...\n")

# Loop 1: Deputados
for index, linha in df_dep.iterrows():
    id_dep = linha['id']
    nome_dep = linha['nome']
    if nome_dep == 'Coronel Armando':
        # Loop 2: Meses (1 a 12)
        for mes in range(1, 13):
            # URL Inicial do mês
            url_despesa = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{id_dep}/despesas?ano=2025&mes={mes}&ordem=ASC&ordenarPor=ano"
            
            pagina_atual = 1 # Contador visual para sabermos a página
            
            # Loop 3: Paginação
            while url_despesa:
                # Feedback visual (printa na mesma linha para não poluir, usando end='\r')
                print(f"Coletando: {nome_dep} | Mês: {mes} | Pág: {pagina_atual}                   ", end='\r')
                
                try:
                    resp = requests.get(url_despesa, timeout=10) # Timeout para não travar para sempre
                    
                    if resp.status_code == 200:
                        dados_json = resp.json()
                        despesas_pagina = dados_json['dados']
                        
                        # Processa os dados
                        if despesas_pagina:
                            for despesa in despesas_pagina:
                                despesa['id_deputado'] = id_dep
                                despesa['nome_deputado'] = nome_dep
                            lista_todas_despesas.extend(despesas_pagina)
                        
                        # Prepara para a próxima página
                        url_despesa = None
                        for link in dados_json['links']:
                            if link['rel'] == 'next':
                                url_despesa = link['href']
                                pagina_atual += 1
                    
                    else:
                        # REGISTRA O ERRO DE STATUS (ex: 404, 500, 429)
                        erro_info = {
                            'id_deputado': id_dep,
                            'nome': nome_dep,
                            'mes': mes,
                            'pagina_tentada': pagina_atual,
                            'erro': f"Status Code {resp.status_code}",
                            'url': url_despesa
                        }
                        relatorio_erros.append(erro_info)
                        print(f"\n[!] Erro {resp.status_code} em {nome_dep} (Mês {mes})")
                        url_despesa = None # Para o loop desse mês e vai pro próximo
                
                except Exception as e:
                    # REGISTRA ERROS DE CONEXÃO/TIMEOUT
                    erro_info = {
                        'id_deputado': id_dep,
                        'nome': nome_dep,
                        'mes': mes,
                        'pagina_tentada': pagina_atual,
                        'erro': str(e),
                        'url': url_despesa
                    }
                    relatorio_erros.append(erro_info)
                    print(f"\n[!] Erro de conexão em {nome_dep} (Mês {mes}): {e}")
                    url_despesa = None # Para o loop desse mês

print("\n" + "="*50)
print("COLETA FINALIZADA!")
print("="*50)

# Criando os DataFrames finais
df_final = pd.DataFrame(lista_todas_despesas)
df_erros = pd.DataFrame(relatorio_erros)

# Mostrando resumo
print(f"Total de registros coletados: {len(df_final)}")
print(f"Total de erros encontrados: {len(df_erros)}")

if not df_erros.empty:
    print("\n--- RELATÓRIO DE ERROS ---")
    print(df_erros.head())
else:
    print("\nSucesso total! Nenhum erro registrado.")

Iniciando coleta de 18 deputados...

Coletando: Coronel Armando | Mês: 12 | Pág: 1                   
COLETA FINALIZADA!
Total de registros coletados: 213
Total de erros encontrados: 0

Sucesso total! Nenhum erro registrado.


In [20]:
df_final.to_csv('Coronel cotas.csv', index=False)

In [ ]:
df_final.to_csv('gastos_deputados.csv', index=False)
df_erros.to_csv('gastos_deputados_erros.csv', index=False)

In [21]:
import requests
import pandas as pd

lista_detalhes = []

print("Coletando fotos e detalhes...")

# Vamos usar o df_deputados apenas para pegar os IDs
for index, linha in df_dep.iterrows():
    id_dep = linha['id']
    
    # URL específica do perfil do deputado
    url = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{id_dep}"
    
    try:
        resp = requests.get(url, timeout=10)
        
        if resp.status_code == 200:
            json_data = resp.json()
            
            # Navegando no JSON: dados -> ultimoStatus -> campos
            status = json_data['dados']['ultimoStatus']
            
            info = {
                'id': id_dep,
                'nome_civil': json_data['dados']['nomeCivil'],
                'nome': status['nome'],           # Nome de urna
                'url': status['urlFoto'],         # A foto
                'partido': status['siglaPartido'] # O partido
            }
            
            lista_detalhes.append(info)
            print(f"Pegou: {status['nome']}")
            
    except Exception as e:
        print(f"Erro ao pegar id {id_dep}")

# Criando o DataFrame final
df_fotos_final = pd.DataFrame(lista_detalhes)

df_fotos_final.head()

Coletando fotos e detalhes...
Pegou: Ana Paula Lima
Pegou: Caroline de Toni
Pegou: Cobalchini
Pegou: Daniel Freitas
Pegou: Daniela Reinehr
Pegou: Fabio Schiochet
Pegou: Geovania de Sá
Pegou: Gilson Marques
Pegou: Ismael
Pegou: Jorge Goetten
Pegou: Julia Zanatta
Pegou: Luiz Fernando Vampiro
Pegou: Pedro Uczai
Pegou: Pezenti
Pegou: Ricardo Guidi
Pegou: Zé Trovão
Pegou: Carlos Chiodini
Pegou: Coronel Armando


,id,nome_civil,nome,url,partido
0,220556,ANA PAULA DE SOUZA LIMA,Ana Paula Lima,https://www.camara.leg.br/internet/deputado/ba...,PT
1,204369,CAROLINE RODRIGUES DE TONI,Caroline de Toni,https://www.camara.leg.br/internet/deputado/ba...,PL
2,98148,VALDIR VITAL COBALCHINI,Cobalchini,https://www.camara.leg.br/internet/deputado/ba...,MDB
3,204367,DANIEL COSTA DE FREITAS,Daniel Freitas,https://www.camara.leg.br/internet/deputado/ba...,PL
4,220557,DANIELA CRISTINA REINEHR,Daniela Reinehr,https://www.camara.leg.br/internet/deputado/ba...,PL


In [ ]:
df_fotos_final.to_csv('Info deputados.csv')

# Deputados

## Holerite

In [3]:
driver = uc.Chrome()
driver.implicitly_wait(10)
wait = WebDriverWait(driver, 10)
driver.get('https://www.camara.leg.br/transparencia/recursos-humanos/funcionarios?search=&categoriaFuncional=Deputado&areaDeAtuacao=&situacao=Em%20exerc%C3%ADcio')

In [4]:
df_dep = pd.read_csv('./Info deputados.csv')

In [6]:
def buscar_nome(nome):
    # Seu código de encontrar o elemento
    busca = driver.find_element(By.CSS_SELECTOR, '#busca')

    # 1. Limpa o campo (usando JS para garantir)
    driver.execute_script("arguments[0].value = '';", busca)

    # 2. Insere o texto com acento via JS
    driver.execute_script(f"arguments[0].value = '{nome}';", busca)

    # 3. (Importante) Dispara o evento de 'input' para o site perceber que houve mudança
    # Alguns sites (React/Vue/Angular) precisam disso para 'ver' o texto digitado
    driver.execute_script("arguments[0].dispatchEvent(new Event('input', { bubbles: true }));", busca)

In [7]:
lista_renumeracao = []

for index, linha in df_dep.iterrows():
    nome = linha['nome_civil']
    if nome == 'LUIZ ARMANDO SCHROEDER REIS':
        print(f'Acessando o depudo {nome}')
        buscar_nome(nome)
        time.sleep(0.4)
        buscar = driver.find_element(By.CSS_SELECTOR, 'div.busca-form__botao > button')
        buscar.click()
        
        time.sleep(2)
        
        link = driver.find_element(By.CSS_SELECTOR, 'section.resultados-busca table tbody > tr > td:last-child > a').get_attribute('href')
        
        lista_renumeracao.append(
            {
                'nome':nome,
                'link': link
            }
        )

Acessando o depudo LUIZ ARMANDO SCHROEDER REIS


In [8]:
lista_renumeracao

[{'nome': 'LUIZ ARMANDO SCHROEDER REIS',
  'link': 'https://www.camara.leg.br/transparencia/recursos-humanos/remuneracao/Rw29VM3boNr29GPA8yEb'}]

In [ ]:
lista_renumeracao[6]['link'] = 'https://www.camara.leg.br/transparencia/recursos-humanos/remuneracao/OLD6Bm94oz96Nd6MvKxz'

[{'nome': 'ANA PAULA DE SOUZA LIMA',
  'link': 'https://www.camara.leg.br/transparencia/recursos-humanos/remuneracao/OLD6Bm94o1aG3ERMvKxz'},
 {'nome': 'CAROLINE RODRIGUES DE TONI',
  'link': 'https://www.camara.leg.br/transparencia/recursos-humanos/remuneracao/OLD6Bm94o14754RMvKxz'},
 {'nome': 'VALDIR VITAL COBALCHINI',
  'link': 'https://www.camara.leg.br/transparencia/recursos-humanos/remuneracao/OLD6Bm94o1ao1eBMvKxz'},
 {'nome': 'DANIEL COSTA DE FREITAS',
  'link': 'https://www.camara.leg.br/transparencia/recursos-humanos/remuneracao/OLD6Bm94oOVwp8RMvKxz'},
 {'nome': 'DANIELA CRISTINA REINEHR',
  'link': 'https://www.camara.leg.br/transparencia/recursos-humanos/remuneracao/OLD6Bm94o1ENqR3MvKxz'},
 {'nome': 'FÁBIO LUIZ SCHIOCHET FILHO',
  'link': 'https://www.camara.leg.br/transparencia/recursos-humanos/remuneracao/OLD6Bm94o1ENqR3MvKxz'},
 {'nome': 'Geovânia de Sá',
  'link': 'https://www.camara.leg.br/transparencia/recursos-humanos/remuneracao/OLD6Bm94oz96Nd6MvKxz'},
 {'nome': 'GILS

In [9]:
df_links = pd.DataFrame(lista_renumeracao)

In [10]:
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

# Supondo que 'html_content' seja a string com o HTML inteiro que você forneceu
# html_content = driver.page_source (ou a variável onde você guardou o HTML)
html_content = StringIO(driver.page_source)
soup = BeautifulSoup(html_content, 'html.parser')

# Lista para guardar os dados de cada folha encontrada
dados_todas_folhas = []

# 1. Encontra todos os blocos que contêm as informações de remuneração
# Cada "remuneracao-funcionario__info" contém um título (h3) e uma tabela
blocos = soup.find_all('div', class_='remuneracao-funcionario__info')

for bloco in blocos:
    # 2. Pega o título/tipo da folha (ex: "022025 – FOLHA NORMAL")
    titulo_elem = bloco.find('h3', class_='remuneracao-funcionario__mes-ano')
    
    # Se não tiver título, pula (segurança)
    if not titulo_elem:
        continue
        
    tipo_folha = titulo_elem.get_text(strip=True)
    
    # 3. Encontra a tabela dentro deste bloco específico
    tabela = bloco.find('table', class_='tabela-responsiva')
    
    if tabela:
        # Dicionário para representar UMA LINHA do DataFrame final
        # Já começamos adicionando o tipo da tabela
        linha_dados = {'Tipo da Folha': tipo_folha}
        
        # 4. Itera sobre as linhas de dados da tabela
        # Usamos a classe 'tabela-responsiva__linha' para pegar só o que importa
        linhas_tabela = tabela.find_all('tr', class_='tabela-responsiva__linha')
        
        for tr in linhas_tabela:
            colunas = tr.find_all('td')
            if len(colunas) >= 2:
                # Coluna 1 vira o CABEÇALHO (chave do dict)
                descricao = colunas[0].get_text(strip=True)
                # Coluna 2 vira o VALOR (valor do dict)
                valor = colunas[1].get_text(strip=True)
                
                # Adiciona ao dicionário
                linha_dados[descricao] = valor
        
        # Adiciona o dicionário processado na lista principal
        dados_todas_folhas.append(linha_dados)

# 5. Cria o DataFrame final
df_remuneracao = pd.DataFrame(dados_todas_folhas)

# Exibe o resultado
display(df_remuneracao)

""


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO
import time

# ---------------------------------------------------------
# 1. PREPARAÇÃO DOS DADOS (SEU CSV)
# ---------------------------------------------------------


# Carregando o CSV
df_deputados = df_links

# Dicionário para converter número do mês em nome (como pedido)
meses_nomes = {
    1: "janeiro", 2: "fevereiro", 3: "março", 4: "abril",
    5: "maio", 6: "junho", 7: "julho", 8: "agosto",
    9: "setembro", 10: "outubro", 11: "novembro", 12: "dezembro"
}

# ---------------------------------------------------------
# 2. CONFIGURAÇÃO DA REQUISIÇÃO (BOAS PRÁTICAS)
# ---------------------------------------------------------
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# Lista para armazenar todos os dados coletados
todos_dados = []

print(f"Iniciando coleta para {len(df_deputados)} deputados...\n")

# ---------------------------------------------------------
# 3. LOOP PRINCIPAL (DEPUTADOS -> MESES)
# ---------------------------------------------------------
for index, row in df_deputados.iterrows():
    nome_deputado = row['nome']
    link_base = row['link']
    
    # Loop pelos meses de 1 a 11
    for mes in range(1, 12):
        ano = 2025
        nome_mes = meses_nomes[mes]
        
        # Constrói a URL final
        url_final = f"{link_base}?ano={ano}&mes={mes}"
        
        print(f"Coletando: {nome_deputado} | {nome_mes}/{ano} ...", end='\r')
        
        try:
            # Faz a requisição
            response = requests.get(url_final, headers=headers, timeout=10)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                
                # Encontra os blocos de remuneração (para pegar Folha Normal e Complementar)
                blocos = soup.find_all('div', class_='remuneracao-funcionario__info')
                
                # Se não achou blocos, pode ser que não tenha dados para esse mês
                if not blocos:
                    # Opcional: Registrar que não houve dados
                    continue
                
                for bloco in blocos:
                    # Pega o Tipo da Folha (ex: "012025 - FOLHA NORMAL")
                    titulo_elem = bloco.find('h3', class_='remuneracao-funcionario__mes-ano')
                    tipo_folha = titulo_elem.get_text(strip=True) if titulo_elem else "N/A"
                    
                    # Procura a tabela dentro do bloco
                    tabela = bloco.find('table', class_='tabela-responsiva')
                    
                    if tabela:
                        # Dicionário base com as colunas fixas pedidas
                        linha_dados = {
                            'Nome': nome_deputado,
                            'Ano': ano,
                            'Mês': nome_mes,
                            'Tipo da Folha': tipo_folha
                        }
                        
                        # Extrai linhas da tabela
                        tr_linhas = tabela.find_all('tr', class_='tabela-responsiva__linha')
                        
                        for tr in tr_linhas:
                            colunas = tr.find_all('td')
                            if len(colunas) >= 2:
                                descricao = colunas[0].get_text(strip=True)
                                valor = colunas[1].get_text(strip=True)
                                
                                # Adiciona ao dicionário (transpondo: descrição vira coluna)
                                linha_dados[descricao] = valor
                        
                        todos_dados.append(linha_dados)
            else:
                print(f"\n[Erro] Status {response.status_code} para {nome_deputado} em {nome_mes}")
                
        except Exception as e:
            print(f"\n[Exceção] Erro ao coletar {nome_deputado} ({nome_mes}): {e}")
        
        # Pausa para ser gentil com o servidor (Boas práticas)
        time.sleep(0.5) 

print("\n\nColeta Finalizada!")

# ---------------------------------------------------------
# 4. GERAÇÃO DO RESULTADO FINAL
# ---------------------------------------------------------
# Cria o DataFrame
df_final = pd.DataFrame(todos_dados)

# Reordena colunas para garantir que Nome, Ano, Mês venham primeiro
colunas_iniciais = ['Nome', 'Ano', 'Mês', 'Tipo da Folha']
# Pega o resto das colunas que foram criadas dinamicamente
colunas_dinamicas = [c for c in df_final.columns if c not in colunas_iniciais]
df_final = df_final[colunas_iniciais + colunas_dinamicas]

# Mostra as primeiras linhas
print(df_final.head())

# Exporta para CSV (opcional)
# df_final.to_csv('remuneracao_deputados_2025.csv', index=False)

In [11]:
df_final['Tipo da Folha'] = df_final['Tipo da Folha'].str.replace(r'^\d+\s*[-–]\s*', '', regex=True)
# Exibe para conferir
print(df_final['Tipo da Folha'].unique())


['FOLHA NORMAL' 'FOLHA DE ADIANTAMENTO GRATIFICAÇÃO NATALINA']


In [12]:
df_final.to_csv('Renumeração Deputados 2025.csv', index=False)

## Pessoal dos Deputados

In [ ]:
df = pd.read_csv('./Info deputados.csv')
df_deputados = df.copy()

### Servidor Em exercicio

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import concurrent.futures
import random
import re
from datetime import datetime
from tqdm import tqdm  # Importação da barra de progresso

# ---------------------------------------------------------
# CONFIGURAÇÕES DE SEGURANÇA E CONEXÃO
# ---------------------------------------------------------
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
]

def get_headers():
    return {
        'User-Agent': random.choice(user_agents),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
        'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7'
    }

# SEUS DADOS DE DEPUTADOS (Exemplo - Substitua pelo seu DF real)
# data = {'id': [161440, 204361], 'nome': ['Pezenti', 'Carlos Chiodini']}
df_deputados = df.copy()

# Sessão global persistente
session = requests.Session()

# ---------------------------------------------------------
# FASE 1: MAPEAMENTO DO GABINETE (FILTRA "EM EXERCÍCIO")
# ---------------------------------------------------------
def mapear_gabinete(dados_deputado):
    id_dep, nome_dep = dados_deputado
    url = f"https://www.camara.leg.br/deputados/{id_dep}/pessoal-gabinete?ano=2025"
    
    funcionarios_encontrados = []
    
    try:
        # Delay de segurança
        time.sleep(random.uniform(2.0, 4.0))
        
        resp = session.get(url, headers=get_headers(), timeout=20)
        
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.content, 'html.parser')
            
            # Procura tabela "Em exercício"
            titulos = soup.find_all(['h2', 'h3'], class_='titulo-secao')
            tabela_alvo = None
            
            for titulo in titulos:
                if "Em exercício" in titulo.get_text():
                    tabela_alvo = titulo.find_next('table')
                    break
            
            if not tabela_alvo:
                tabela_alvo = soup.find('table', class_='table')

            if tabela_alvo:
                tbody = tabela_alvo.find('tbody')
                if tbody:
                    rows = tbody.find_all('tr')
                    for row in rows:
                        cols = row.find_all('td')
                        if len(cols) >= 4:
                            nome_servidor = cols[0].get_text(strip=True)
                            cargo = cols[2].get_text(strip=True) if len(cols) > 2 else "N/A"
                            periodo_texto = cols[3].get_text(strip=True)
                            
                            # Lógica Inteligente de Data
                            mes_inicio = 1
                            match_data = re.search(r'(\d{2}/\d{2}/\d{4})', periodo_texto)
                            if match_data:
                                try:
                                    data_inicio = datetime.strptime(match_data.group(1), '%d/%m/%Y')
                                    if data_inicio.year == 2025:
                                        mes_inicio = data_inicio.month
                                    elif data_inicio.year > 2025:
                                        continue 
                                except:
                                    pass 
                            
                            link_tag = cols[-1].find('a')
                            if link_tag and 'href' in link_tag.attrs:
                                link_base = link_tag['href']
                                
                                funcionarios_encontrados.append({
                                    'Nome Deputado': nome_dep,
                                    'ID Deputado': id_dep,
                                    'Nome Servidor': nome_servidor,
                                    'Cargo': cargo,
                                    'Link Base': link_base,
                                    'Mes Inicio': mes_inicio
                                })
                                
    except Exception as e:
        # Erros silenciosos na fase 1 para não poluir a barra de progresso
        pass 
        
    return funcionarios_encontrados

# ---------------------------------------------------------
# FASE 2: BAIXAR HOLERITE (COM TQDM E BACKOFF)
# ---------------------------------------------------------
def processar_holerite(task):
    funcionario, mes, ano = task
    url_final = f"{funcionario['Link Base']}?ano={ano}&mes={mes}"
    dados_extraidos = []
    
    max_tentativas = 3
    
    for tentativa in range(max_tentativas):
        try:
            # Delay ALTO para evitar ban (3 a 6 segundos)
            time.sleep(random.uniform(3.0, 6.0))
            
            resp = session.get(url_final, headers=get_headers(), timeout=25)
            
            # Backoff se tomar Rate Limit (429)
            if resp.status_code == 429:
                wait_time = 60 + (tentativa * 30)
                # O tqdm pode quebrar se dermos print normal, então usamos tqdm.write
                tqdm.write(f"!!! RATE LIMIT (429). Pausando {wait_time}s... ({funcionario['Nome Servidor']})")
                time.sleep(wait_time)
                continue 
            
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.content, 'html.parser')
                blocos = soup.find_all('div', class_='remuneracao-funcionario__info')
                
                if not blocos:
                    break 
                
                for bloco in blocos:
                    titulo_elem = bloco.find('h3', class_='remuneracao-funcionario__mes-ano')
                    raw_titulo = titulo_elem.get_text(strip=True) if titulo_elem else "FOLHA DESCONHECIDA"
                    tipo_limpo = re.sub(r'^\d+\s*[-–]\s*', '', raw_titulo)
                    
                    tabela = bloco.find('table', class_='tabela-responsiva')
                    if tabela:
                        linha = {
                            'Nome Deputado': funcionario['Nome Deputado'],
                            'Nome Servidor': funcionario['Nome Servidor'],
                            'Cargo': funcionario['Cargo'],
                            'Ano': ano,
                            'Mês': mes,
                            'Tipo Folha': tipo_limpo
                        }
                        
                        trs = tabela.find_all('tr', class_='tabela-responsiva__linha')
                        for tr in trs:
                            tds = tr.find_all('td')
                            if len(tds) >= 2:
                                desc = tds[0].get_text(strip=True)
                                val = tds[1].get_text(strip=True)
                                linha[desc] = val
                                
                        dados_extraidos.append(linha)
                break 
                
        except Exception as e:
            time.sleep(5)
            
    return dados_extraidos

# =========================================================
# ORQUESTRAÇÃO PRINCIPAL
# =========================================================

if __name__ == "__main__":
    
    print(f"--- FASE 1: Mapeando {len(df_deputados)} gabinetes (Modo Seguro) ---")
    lista_servidores = []
    
    # Barra de progresso para os Gabinetes
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        args_deputados = [(row['id'], row['nome']) for _, row in df_deputados.iterrows()]
        
        # Envolvendo o executor map com tqdm
        resultados = list(tqdm(executor.map(mapear_gabinete, args_deputados), total=len(args_deputados), desc="Mapeando Gabinetes"))
        
        for res in resultados:
            lista_servidores.extend(res)
            
    print(f"Total de servidores ativos encontrados: {len(lista_servidores)}")
    
    # --- PREPARAÇÃO DA FILA INTELIGENTE ---
    tarefas = []
    for serv in lista_servidores:
        # Gera tarefas apenas do mês de admissão em diante
        for mes in range(serv['Mes Inicio'], 13): 
            tarefas.append((serv, mes, 2025))
            
    total_tasks = len(tarefas)
    print(f"Total de requisições necessárias: {total_tasks}")
    print("--- FASE 2: Baixando holerites ---")
    
    dados_finais = []
    
    # --- EXECUÇÃO SEGURA COM BARRA DE PROGRESSO ---
    # max_workers=3 é lento, mas é o que evita o BAN
    with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
        futures = [executor.submit(processar_holerite, t) for t in tarefas]
        
        # TQDM entra aqui para monitorar o as_completed
        for future in tqdm(concurrent.futures.as_completed(futures), total=total_tasks, desc="Baixando Folhas", unit="req"):
            resultado = future.result()
            if resultado:
                dados_finais.extend(resultado)

    print("\nColeta Finalizada!")
    
    # --- EXPORTAÇÃO ---
    if dados_finais:
        df_holerites = pd.DataFrame(dados_finais)
        
        cols_fixas = ['Nome Deputado', 'Nome Servidor', 'Cargo', 'Ano', 'Mês', 'Tipo Folha']
        cols_extras = [c for c in df_holerites.columns if c not in cols_fixas]
        df_holerites = df_holerites[cols_fixas + cols_extras]
        
        print(f"Total de registros obtidos: {len(df_holerites)}")
        # df_holerites.to_csv('remuneracao_final_2025.csv', index=False) # Descomente para salvar
        print(df_holerites.head())
    else:
        print("Nenhum dado foi coletado.")

In [ ]:
print("--- INICIANDO REPESCAGEM (PENTE FINO) ---")

# 1. Cria um conjunto (set) com os pares (Nome, Mês) que faltaram
# Ignoramos o mês 12 pois você já confirmou que não existe
pares_faltantes = set(
    zip(faltantes[faltantes['Mês'] != 12]['Nome Servidor'], 
        faltantes[faltantes['Mês'] != 12]['Mês'])
)

print(f"Total de itens para tentar baixar novamente: {len(pares_faltantes)}")

# 2. Reconstrói a lista de tarefas apenas para quem falhou
tarefas_retry = []
for t in tarefas:
    servidor = t[0]
    mes = t[1]
    
    # Se esse par (servidor, mes) está na lista de faltantes, adiciona na repescagem
    if (servidor['Nome Servidor'], mes) in pares_faltantes:
        tarefas_retry.append(t)

# 3. Executa a repescagem
dados_resgatados = []

if tarefas_retry:
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        futures = [executor.submit(processar_holerite, t) for t in tarefas_retry]
        
        # Barra de progresso da repescagem
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(tarefas_retry), desc="Repescando"):
            resultado = future.result()
            if resultado:
                dados_resgatados.extend(resultado)

    print(f"\nRecuperados na repescagem: {len(dados_resgatados)}")

    # 4. Junta os dados novos com os dados antigos
    if dados_resgatados:
        df_resgatados = pd.DataFrame(dados_resgatados)
        # Garante as mesmas colunas
        cols_extras = [c for c in df_resgatados.columns if c not in cols_fixas]
        df_resgatados = df_resgatados[cols_fixas + cols_extras]
        
        # Concatena
        df_holerites = pd.concat([df_holerites, df_resgatados], ignore_index=True)
        print(f"Total FINAL de registros: {len(df_holerites)}")

else:
    print("Nada para repescar (todos os faltantes eram do mês 12).")

# 5. Salva o arquivo final consolidado
df_holerites.to_csv("holerites_consolidado_2025.csv", index=False)

--- INICIANDO REPESCAGEM (PENTE FINO) ---
Total de itens para tentar baixar novamente: 121


Repescando: 100%|██████████| 150/150 [12:05<00:00,  4.84s/it]


Recuperados na repescagem: 26
Total FINAL de registros: 3011


In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import concurrent.futures
import random
import re
from datetime import datetime
from tqdm import tqdm
import threading
import os
import subprocess # Necessário para rodar comandos do Windows

# ---------------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------------
# Headers rotativos
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0'
]

# --- VERIFIQUE ESTE CAMINHO NO SEU PC ---
# Geralmente fica em "C:\Program Files\Proton\VPN\ProtonVPN.exe"
# Se não funcionar, clique com botão direito no atalho do Proton > Propriedades > Copie o "Destino"
CAMINHO_PROTON = r"C:\Program Files\Proton\VPN\ProtonVPN.exe"

def get_headers():
    return {
        'User-Agent': random.choice(user_agents),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    }

session = requests.Session()

# Trava para impedir que várias threads tentem reiniciar o VPN ao mesmo tempo
input_lock = threading.Lock()

# SEUS DADOS (Substitua df_deputados pelo seu DF real carregado)
# df_deputados = ... 

# ---------------------------------------------------------
# FUNÇÃO PARA REINICIAR O PROTON (OPÇÃO 3 AUTOMATIZADA)
# ---------------------------------------------------------
def reiniciar_proton_app():
    print("🔄 Reiniciando aplicativo ProtonVPN...")
    
    # 1. Mata o processo do ProtonVPN (Força o fechamento)
    # stdout=subprocess.DEVNULL esconde a mensagem de sucesso do terminal para manter limpo
    subprocess.run(["taskkill", "/F", "/IM", "ProtonVPN.exe"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    time.sleep(3) # Espera o Windows liberar o processo
    
    # 2. Abre o aplicativo novamente
    try:
        subprocess.Popen(CAMINHO_PROTON)
        print("⏳ Aguardando 25 segundos para conexão automática...")
        # O app demora para abrir e conectar. Ajuste esse tempo se sua internet for lenta.
        time.sleep(25) 
        print("✅ VPN Reiniciada! Retomando coleta...")
        return True
    except FileNotFoundError:
        print(f"❌ ERRO: Não achei o ProtonVPN em: {CAMINHO_PROTON}")
        print("Verifique o caminho no início do script.")
        return False

# ---------------------------------------------------------
# FUNÇÃO DE RESGATE (AGORA AUTOMÁTICA)
# ---------------------------------------------------------
def esperar_novo_ip(erro_msg):
    # Adquire a trava. Se 5 threads bloquearem ao mesmo tempo, 
    # só a primeira entra aqui e reinicia a VPN. As outras esperam a VPN voltar.
    with input_lock:
        # Verifica se o erro foi recente (evita reiniciar 2x seguidas sem querer)
        tqdm.write(f"\n🚨 BLOQUEIO DETECTADO ({erro_msg}). Iniciando troca de IP automática...")
        
        # Chama a função que mata e abre o app
        reiniciar_proton_app()
        
        # Renova a sessão para limpar cookies antigos que podem denunciar o bot
        global session
        session = requests.Session()

# ---------------------------------------------------------
# FASE 2: BAIXAR HOLERITE (COM PAUSA AUTOMÁTICA)
# ---------------------------------------------------------
def processar_holerite(task):
    funcionario, mes, ano = task
    url_final = f"{funcionario['Link Base']}?ano={ano}&mes={mes}"
    dados_extraidos = []
    
    # Loop Infinito: Só sai se tiver sucesso ou certeza que não existe
    while True:
        try:
            # Delay aleatório
            time.sleep(random.uniform(2.0, 4.0))
            
            resp = session.get(url_final, headers=get_headers(), timeout=20)
            
            # BLOQUEIO DETECTADO (429 = Rate Limit, 403 = Forbidden)
            if resp.status_code in [429, 403]:
                esperar_novo_ip(f"Status Code {resp.status_code}")
                continue # Volta para o início do While e tenta de novo com IP novo
            
            # SUCESSO
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.content, 'html.parser')
                
                # Verifica se é uma página de erro disfarçada
                texto_pagina = soup.get_text().lower()
                if "access denied" in texto_pagina or "request rejected" in texto_pagina:
                    esperar_novo_ip("Soft Block (Access Denied na página)")
                    continue

                blocos = soup.find_all('div', class_='remuneracao-funcionario__info')
                
                if not blocos:
                    # Página vazia legítima
                    break 
                
                for bloco in blocos:
                    titulo_elem = bloco.find('h3', class_='remuneracao-funcionario__mes-ano')
                    raw_titulo = titulo_elem.get_text(strip=True) if titulo_elem else "FOLHA"
                    tipo_limpo = re.sub(r'^\d+\s*[-–]\s*', '', raw_titulo)
                    
                    tabela = bloco.find('table', class_='tabela-responsiva')
                    if tabela:
                        linha = {
                            'Nome Deputado': funcionario['Nome Deputado'],
                            'Nome Servidor': funcionario['Nome Servidor'],
                            'Cargo': funcionario['Cargo'],
                            'Período Contrato': funcionario.get('Periodo Texto', 'Ativo'),
                            'Ano': ano,
                            'Mês': mes,
                            'Tipo Folha': tipo_limpo
                        }
                        
                        trs = tabela.find_all('tr', class_='tabela-responsiva__linha')
                        for tr in trs:
                            tds = tr.find_all('td')
                            if len(tds) >= 2:
                                desc = tds[0].get_text(strip=True)
                                val = tds[1].get_text(strip=True)
                                linha[desc] = val
                                
                        dados_extraidos.append(linha)
                break # Sai do loop while
            
            # Outros erros (500, 404) -> Segue o baile
            else:
                break

        except Exception as e:
            # Erro de conexão (internet caiu enquanto o Proton reiniciava ou falhou)
            # Se der erro de conexão, assume que a VPN caiu e tenta reiniciar de novo
            esperar_novo_ip(f"Erro de Conexão: {e}")
            continue 
            
    return dados_extraidos

# =========================================================
# ORQUESTRAÇÃO 
# =========================================================
if __name__ == "__main__":
    
    # 2. Criação da Fila
    print("Gerando fila de tarefas...")
    tarefas = []
    
    # OPÇÃO B: Usar a lista 'tarefas_retry' (Se existir)
    if 'tarefas_retry' in locals() and tarefas_retry:
        print("Usando lista de repescagem (tarefas que falharam antes).")
        tarefas = tarefas_retry
    # OPÇÃO A: Se não tiver lista de retry, descomente aqui para rodar tudo se necessário
    elif 'lista_servidores' in locals():
         print("Gerando lista completa a partir dos servidores mapeados...")
         for serv in lista_servidores:
             # Usa .get() para garantir compatibilidade com scripts anteriores
             fim = serv.get('Mes Fim', 12) 
             for mes in range(serv['Mes Inicio'], fim + 1): 
                 tarefas.append((serv, mes, 2025))
    else:
        print("ATENÇÃO: Nenhuma lista de tarefas encontrada.")
            
    print(f"Total de requisições na fila: {len(tarefas)}")
    
    dados_finais = []
    
    # --- EXECUÇÃO IMORTAL ---
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(processar_holerite, t) for t in tarefas]
        
        contador_save = 0
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(tarefas), desc="Baixando (Auto-VPN)", unit="req"):
            resultado = future.result()
            if resultado:
                dados_finais.extend(resultado)
            
            # --- CHECKPOINT AUTOMÁTICO ---
            contador_save += 1
            if contador_save % 500 == 0:
                pd.DataFrame(dados_finais).to_csv("backup_parcial.csv", index=False)
                tqdm.write(f"Checkpoint salvo! ({contador_save} requisições processadas)")

    # 4. DataFrame Final
    if dados_finais:
        df_holerites = pd.DataFrame(dados_finais)
        
        cols_fixas = ['Nome Deputado', 'Nome Servidor', 'Cargo', 'Ano', 'Mês', 'Tipo Folha', 'Período Contrato']
        cols_existentes = [c for c in cols_fixas if c in df_holerites.columns]
        cols_dinamicas = [c for c in df_holerites.columns if c not in cols_existentes]
        
        df_holerites = df_holerites[cols_existentes + cols_dinamicas]
        
        print("\nSucesso Total!")
        print(f"Total de linhas coletadas: {len(df_holerites)}")
        
        df_holerites.to_csv("holerites_COMPLETO_2025.csv", index=False)
        print("Arquivo 'holerites_COMPLETO_2025.csv' salvo.")
    else:
        print("\nNenhum dado encontrado.")

Gerando fila de tarefas...
Usando lista de repescagem (tarefas que falharam antes).
Total de requisições na fila: 150


Baixando (Auto-VPN): 100%|██████████| 150/150 [01:53<00:00,  1.33req/s]


Sucesso Total!
Total de linhas coletadas: 26
Arquivo 'holerites_COMPLETO_2025.csv' salvo.


### Servidor No histórico

In [5]:
df=pd.read_csv("Info deputados.csv")

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import concurrent.futures
import random
import re
from datetime import datetime
from tqdm import tqdm
import threading
import os
import subprocess  # Necessário para reiniciar o app

# ---------------------------------------------------------
# CONFIGURAÇÕES E CAMINHOS
# ---------------------------------------------------------
# Headers rotativos
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
]

# CAMINHO EXATO DO PROTON (Baseado nos seus testes)
CAMINHO_PROTON = r"C:\Program Files\Proton\VPN\ProtonVPN.Launcher.exe"

def get_headers():
    return {
        'User-Agent': random.choice(user_agents),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    }

session = requests.Session()
input_lock = threading.Lock() # Trava para impedir que várias threads reiniciem a VPN juntas

# Substitua pelo seu DF real de deputados se já estiver carregado
df_deputados = df

# ---------------------------------------------------------
# FUNÇÃO PARA REINICIAR O PROTON (OPÇÃO AUTOMÁTICA)
# ---------------------------------------------------------
def reiniciar_proton_app():
    tqdm.write("\n🔄 Reiniciando aplicativo ProtonVPN...")
    
    # 1. Mata os processos específicos do Proton (Client e Launcher)
    # Ocultamos a saída (stdout/stderr) para não poluir o terminal
    subprocess.run("taskkill /F /IM ProtonVPN.Client.exe /T", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run("taskkill /F /IM ProtonVPN.Launcher.exe /T", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    time.sleep(5) # Espera o Windows liberar a memória
    
    # 2. Abre o aplicativo novamente
    try:
        if os.path.exists(CAMINHO_PROTON):
            subprocess.Popen(CAMINHO_PROTON)
            
            # TEMPO DE ESPERA AJUSTADO: 25 Segundos
            # (15s para conectar + 10s de margem de segurança)
            tqdm.write("⏳ Aguardando 25 segundos para conexão automática...")
            time.sleep(15) 
            
            tqdm.write("✅ VPN Reiniciada! Retomando coleta...")
            return True
        else:
            tqdm.write(f"❌ ERRO CRÍTICO: Arquivo não encontrado em: {CAMINHO_PROTON}")
            return False
    except Exception as e:
        tqdm.write(f"❌ Erro ao abrir Proton: {e}")
        return False

# ---------------------------------------------------------
# FUNÇÃO DE RESGATE (GERENCIA O BLOQUEIO)
# ---------------------------------------------------------
def esperar_novo_ip(erro_msg):
    # Adquire a trava. Se 10 threads falharem ao mesmo tempo, só a primeira reinicia a VPN.
    # As outras esperam a VPN voltar.
    with input_lock:
        # Verifica se o erro foi recente (evita reiniciar 2x seguidas se a VPN já tiver voltado)
        tqdm.write(f"🚨 BLOQUEIO DETECTADO ({erro_msg}). Iniciando troca de IP...")
        
        reiniciar_proton_app()
        
        # Renova a sessão para limpar cookies antigos
        global session
        session = requests.Session()

# ---------------------------------------------------------
# FASE 1: MAPEAMENTO (FOCO NO HISTÓRICO 2025)
# ---------------------------------------------------------
def mapear_historico_gabinete(dados_deputado):
    id_dep, nome_dep = dados_deputado
    url = f"https://www.camara.leg.br/deputados/{id_dep}/pessoal-gabinete?ano=2025"
    
    funcionarios_encontrados = []
    
    try:
        # Delay pequeno na fase 1 (raramente bloqueia aqui)
        time.sleep(random.uniform(1.0, 3.0))
        resp = session.get(url, headers=get_headers(), timeout=20)
        
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.content, 'html.parser')
            
            # Procura especificamente o título "Histórico"
            titulos = soup.find_all(['h2', 'h3'], class_='titulo-secao')
            tabela_historico = None
            
            for titulo in titulos:
                if "Histórico" in titulo.get_text():
                    tabela_historico = titulo.find_next('table')
                    break
            
            if tabela_historico:
                tbody = tabela_historico.find('tbody')
                if tbody:
                    rows = tbody.find_all('tr')
                    for row in rows:
                        cols = row.find_all('td')
                        if len(cols) >= 4:
                            nome_servidor = cols[0].get_text(strip=True)
                            periodo_texto = cols[3].get_text(strip=True)
                            
                            # Regex para capturar as duas datas
                            match_datas = re.search(r'De (\d{2}/\d{2}/\d{4}) a (\d{2}/\d{2}/\d{4})', periodo_texto)
                            
                            if match_datas:
                                try:
                                    dt_inicio = datetime.strptime(match_datas.group(1), '%d/%m/%Y')
                                    dt_fim = datetime.strptime(match_datas.group(2), '%d/%m/%Y')
                                    
                                    # Filtro de Ano: Verifica se cruza com 2025
                                    ano_target = 2025
                                    if dt_fim.year < ano_target or dt_inicio.year > ano_target:
                                        continue 
                                    
                                    # Define Intervalo de Meses
                                    mes_start = 1
                                    if dt_inicio.year == ano_target:
                                        mes_start = dt_inicio.month
                                        
                                    mes_end = 12
                                    if dt_fim.year == ano_target:
                                        mes_end = dt_fim.month
                                    
                                    link_tag = cols[-1].find('a')
                                    if link_tag and 'href' in link_tag.attrs:
                                        funcionarios_encontrados.append({
                                            'Nome Deputado': nome_dep,
                                            'Nome Servidor': nome_servidor,
                                            'Cargo': cols[2].get_text(strip=True),
                                            'Periodo Texto': periodo_texto,
                                            'Link Base': link_tag['href'],
                                            'Mes Inicio': mes_start,
                                            'Mes Fim': mes_end
                                        })
                                except:
                                    pass
    except Exception as e:
        pass
        
    return funcionarios_encontrados

# ---------------------------------------------------------
# FASE 2: BAIXAR HOLERITE (COM AUTO-VPN)
# ---------------------------------------------------------
def processar_holerite(task):
    funcionario, mes, ano = task
    url_final = f"{funcionario['Link Base']}?ano={ano}&mes={mes}"
    dados_extraidos = []
    
    # Loop Infinito: Tenta até conseguir ou até confirmar que a página é vazia
    while True:
        try:
            # TURBO MODE: Delay reduzido pois temos VPN automática
            time.sleep(random.uniform(1.0, 2.5))
            
            resp = session.get(url_final, headers=get_headers(), timeout=25)
            
            # BLOQUEIO DETECTADO
            if resp.status_code in [429, 403]:
                esperar_novo_ip(f"Status {resp.status_code}")
                continue # Volta pro início do while com IP novo
            
            # SUCESSO (200 OK)
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.content, 'html.parser')
                
                # Check de soft-block (página de erro camuflada)
                texto_pagina = soup.get_text().lower()
                if "access denied" in texto_pagina or "request rejected" in texto_pagina:
                    esperar_novo_ip("Soft Block (Texto na página)")
                    continue

                blocos = soup.find_all('div', class_='remuneracao-funcionario__info')
                
                # Se não tem blocos, é um mês vazio legítimo (ex: funcionário não recebeu naquele mês)
                if not blocos: 
                    break 
                
                for bloco in blocos:
                    titulo_elem = bloco.find('h3', class_='remuneracao-funcionario__mes-ano')
                    raw_titulo = titulo_elem.get_text(strip=True) if titulo_elem else "FOLHA"
                    tipo_limpo = re.sub(r'^\d+\s*[-–]\s*', '', raw_titulo)
                    
                    tabela = bloco.find('table', class_='tabela-responsiva')
                    if tabela:
                        linha = {
                            'Nome Deputado': funcionario['Nome Deputado'],
                            'Nome Servidor': funcionario['Nome Servidor'],
                            'Cargo': funcionario['Cargo'],
                            'Período Contrato': funcionario['Periodo Texto'],
                            'Ano': ano,
                            'Mês': mes,
                            'Tipo Folha': tipo_limpo
                        }
                        
                        trs = tabela.find_all('tr', class_='tabela-responsiva__linha')
                        for tr in trs:
                            tds = tr.find_all('td')
                            if len(tds) >= 2:
                                desc = tds[0].get_text(strip=True)
                                val = tds[1].get_text(strip=True)
                                linha[desc] = val
                        dados_extraidos.append(linha)
                break # Sai do loop (Sucesso)
            
            # Outros erros (404/500) -> Ignora e segue para o próximo mês
            else:
                break 

        except Exception as e:
            # Erro de conexão (VPN caiu, internet oscilou)
            # Aciona a troca de IP para garantir que a conexão volte limpa
            esperar_novo_ip(f"Erro Conexão: {e}")
            continue # Tenta de novo
            
    return dados_extraidos

# =========================================================
# ORQUESTRAÇÃO
# =========================================================
if __name__ == "__main__":
    
    # Se você já tiver a lista de servidores na memória, pode pular essa parte
    if 'lista_servidores' not in locals() or not lista_servidores:
        print(f"--- FASE 1: Mapeando HISTÓRICO 2025 de {len(df_deputados)} gabinetes ---")
        lista_servidores = []
        
        # Mapeamento com 3 threads (Seguro)
        with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
            args = [(row['id'], row['nome']) for _, row in df_deputados.iterrows()]
            resultados = list(tqdm(executor.map(mapear_historico_gabinete, args), total=len(args), desc="Lendo Históricos"))
            
            for res in resultados:
                lista_servidores.extend(res)
        print(f"Servidores encontrados no histórico: {len(lista_servidores)}")
    else:
        print(f"Usando {len(lista_servidores)} servidores já carregados na memória.")
    
    # 2. Criação da Fila
    print("Gerando fila de downloads...")
    tarefas = []
    
    # OPÇÃO: Usar Repescagem (Se você rodou a auditoria antes)
    if 'tarefas_retry' in locals() and tarefas_retry:
        print(">>> MODO REPESCAGEM ATIVADO (Baixando apenas faltantes) <<<")
        tarefas = tarefas_retry
    else:
        # Modo Normal (Baixa tudo da lista mapeada)
        for serv in lista_servidores:
            for mes in range(serv['Mes Inicio'], serv['Mes Fim'] + 1): 
                tarefas.append((serv, mes, 2025))
            
    print(f"Total de requisições na fila: {len(tarefas)}")
    
    # 3. Download (Fase Turbo com Auto-VPN)
    dados_finais = []
    
    # Aumentei para 12 workers já que a VPN resolve bloqueios
    with concurrent.futures.ThreadPoolExecutor(max_workers=15) as executor:
        futures = [executor.submit(processar_holerite, t) for t in tarefas]
        
        contador_save = 0
        # tqdm monitora o progresso real
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(tarefas), desc="Baixando (Auto-VPN)", unit="req"):
            resultado = future.result()
            if resultado:
                dados_finais.extend(resultado)
                
            # Checkpoint a cada 500 requisições
            contador_save += 1
            if contador_save % 500 == 0:
                pd.DataFrame(dados_finais).to_csv("backup_historico_parcial.csv", index=False)

    # 4. DataFrame Final e Salvamento
    if dados_finais:
        df_historico = pd.DataFrame(dados_finais)
        
        # Organização de colunas
        cols_fixas = ['Nome Deputado', 'Nome Servidor', 'Cargo', 'Ano', 'Mês', 'Tipo Folha', 'Período Contrato']
        cols_existentes = [c for c in cols_fixas if c in df_historico.columns]
        cols_dinamicas = [c for c in df_historico.columns if c not in cols_existentes]
        
        df_historico = df_historico[cols_existentes + cols_dinamicas]
        
        print("\nSucesso!")
        print(f"Total de linhas coletadas: {len(df_historico)}")
        
        nome_arquivo = "historico_gabinete_COMPLETO_2025.csv"
        df_historico.to_csv(nome_arquivo, index=False)
        print(f"Arquivo salvo como: {nome_arquivo}")
    else:
        print("\nNenhum dado encontrado.")

--- FASE 1: Mapeando HISTÓRICO 2025 de 17 gabinetes ---


Lendo Históricos: 100%|██████████| 17/17 [00:23<00:00,  1.38s/it]


Servidores encontrados no histórico: 398
Gerando fila de downloads...
Total de requisições na fila: 1297


Baixando (Auto-VPN): 100%|██████████| 1297/1297 [03:24<00:00,  6.34req/s]


Sucesso!
Total de linhas coletadas: 1324
Arquivo salvo como: historico_gabinete_COMPLETO_2025.csv


### Removendo duplicatas

In [4]:
import pandas as pd

df_atual = pd.read_csv('Dados\Deputados\Holerite pessoal\holerite_pessoal_atual.csv')
df_historico = pd.read_csv('Dados\Deputados\Holerite pessoal\holerite_pessoal_historico.csv')

In [9]:
df_atual['Tipo_funcionario'] = 'Atual'
df_historico['Tipo_funcionario'] = 'Temporario'

dfs = [df_atual, df_historico]

df_final = pd.concat(dfs)
df_final

,Nome Deputado,Nome Servidor,Cargo,Ano,Mês,Tipo Folha,a - Remuneração Fixa,b - Vantagens de Natureza Pessoal,a - Função ou Cargo em Comissão,b - Gratificação Natalina,...,a - Abono Permanência,a - Redutor Constitucional,b - Contribuição Previdenciária,c - Imposto de Renda,a - Remuneração após Descontos Obrigatórios,a - Diárias,b - Auxílios,c - Vantagens Indenizatórias,Tipo_funcionario,Período Contrato
0,Ana Paula Lima,CEDENIR ALBERTO SIMON,SP24,2025,3,FOLHA NORMAL,0,0,"17.273,26",0,...,0,0,"-951,62","-3.592,45","12.729,19",0,"1.494,40",0,Atual,NaN
1,Ana Paula Lima,CEDENIR ALBERTO SIMON,SP24,2025,4,FOLHA NORMAL,0,0,"17.273,26",0,...,0,0,"-951,62","-3.592,45","12.729,19",0,"1.462,75",0,Atual,NaN
2,Ana Paula Lima,ANDRÉ RAMOS RIZELO,SP05,2025,11,FOLHA NORMAL,0,0,"3.891,58",0,...,0,0,"-360,39","-50,20","3.480,99",0,"5.497,78",0,Atual,NaN
3,Ana Paula Lima,CEDENIR ALBERTO SIMON,SP24,2025,2,FOLHA NORMAL,0,0,"17.803,68",0,...,0,0,"-951,62","-3.738,32","13.113,74",0,"1.393,11",0,Atual,NaN
4,Ana Paula Lima,ANDRÉ RAMOS RIZELO,SP05,2025,10,FOLHA NORMAL,0,0,"2.205,22",0,...,0,0,"-175,69",0,"2.029,53",0,"1.054,43",0,Atual,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,Carlos Chiodini,VINÍCIUS JOSÉ DE OLIVEIRA CARVALHO,SP14,2025,2,FOLHA NORMAL,0,0,"6.896,27",0,...,0,0,"-775,06","-787,33","5.333,88",0,"1.393,11",0,Temporario,De 17/07/2024 a 03/02/2025
1320,Carlos Chiodini,VIRLEI EDSON DA COSTA,SP24,2025,11,FOLHA NORMAL,0,0,"17.273,26",0,...,0,0,"-951,62","-3.579,72","12.741,92",0,"1.784,42",0,Temporario,De 12/11/2025 a 17/11/2025
1321,Carlos Chiodini,VIRLEI EDSON DA COSTA,SP19,2025,3,FOLHA NORMAL,0,0,"3.445,96","2.396,65",...,0,0,"-499,83","-89,42","5.253,36",0,"364,09",0,Temporario,De 04/02/2025 a 09/03/2025
1322,Carlos Chiodini,VIRLEI EDSON DA COSTA,SP24,2025,1,FOLHA NORMAL,0,0,"16.275,56",0,...,0,0,"-951,62","-3.318,08","12.005,86",0,"1.393,11",0,Temporario,De 07/02/2024 a 03/02/2025


In [10]:
# Lista com as colunas mostradas na imagem
colunas_para_verificar = [
    'Nome Deputado', 
    'Nome Servidor', 
    'Ano', 
    'Mês', 
    'Tipo Folha', 
    'a - Remuneração Fixa', 
    'b - Vantagens de Natureza Pessoal', 
    'a - Função ou Cargo em Comissão'
]

# Remove duplicatas baseadas apenas nessas colunas
df_final = df_final.drop_duplicates(subset=colunas_para_verificar)

In [12]:
df_final.to_csv('Holerite pessoial sem duplicadas.csv')

## Missao Especial

In [18]:
from selenium.webdriver.support.ui import Select

In [14]:
driver = uc.Chrome()
driver.implicitly_wait(10)
wait = WebDriverWait(driver, 10)
driver.get('https://www.camara.leg.br/missao-oficial/')

In [ ]:
select_dep =Select(driver.find_element(By.CSS_SELECTOR, 'select[name="nome-deputado"]'))
select_dep.select_by_visible_text(nome)

NoSuchElementException: Message: Could not locate element with visible text: JORGE GOETTEN; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception


In [29]:
df = pd.read_csv('./Info deputados.csv')

In [ ]:
https://www.camara.leg.br/deputados/178966/auxilio-moradia?ano=2025
https://www.camara.leg.br/deputados/178966/missao-oficial?ano=2025

In [45]:
with open('html gabinete.txt', 'w', encoding='utf-8') as f:
    f.write(driver.page_source)

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# ---------------------------------------------------------
# 1. Configuração
# ---------------------------------------------------------
# Headers para simular um navegador real (evita bloqueios)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# Lista para armazenar todos os dados
lista_viagens = []

print(f"Iniciando coleta de viagens oficiais (2025) para {len(df)} deputados...\n")

# ---------------------------------------------------------
# 2. Loop Principal
# ---------------------------------------------------------
for index, row in df.iterrows():
    id_dep = row['id']
    nome_dep = row['nome']
    
    # Monta a URL dinâmica
    url = f"https://www.camara.leg.br/deputados/{id_dep}/missao-oficial?ano=2025"
    
    print(f"Verificando: {nome_dep} (ID: {id_dep})...", end='\r')
    
    try:
        # Faz a requisição
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Procura a tabela específica de viagens
            # No HTML fornecido, ela tem as classes: "table table-striped table-bordered"
            tabela = soup.find('table', class_='table table-striped table-bordered')
            
            if tabela:
                # Pega o corpo da tabela
                tbody = tabela.find('tbody')
                
                if tbody:
                    linhas = tbody.find_all('tr')
                    
                    for tr in linhas:
                        cols = tr.find_all('td')
                        
                        # Verifica se tem o número certo de colunas para evitar erros
                        if len(cols) >= 6:
                            periodo = cols[0].get_text(strip=True)
                            assunto = cols[1].get_text(strip=True)
                            destino = cols[2].get_text(strip=True)
                            passagens = cols[3].get_text(strip=True)
                            diarias = cols[4].get_text(strip=True)
                            
                            # Extrai o link de detalhes (coluna Relatório)
                            link_elem = cols[5].find('a')
                            link_detalhes = link_elem['href'] if link_elem else None
                            
                            # Se o link for relativo (começar com /), completa ele
                            if link_detalhes and link_detalhes.startswith('/'):
                                link_detalhes = "https://www.camara.leg.br" + link_detalhes
                            
                            # Salva os dados
                            lista_viagens.append({
                                'ID Deputado': id_dep,
                                'Nome Deputado': nome_dep,
                                'Período': periodo,
                                'Assunto': assunto,
                                'Destino': destino,
                                'Passagens (R$)': passagens,
                                'Diárias (R$)': diarias,
                                'Link Detalhes': link_detalhes
                            })
            else:
                # Caso comum: Deputado não viajou ou a tabela não existe na página
                pass 
                
        else:
            print(f"\n[Erro] Status {response.status_code} para {nome_dep}")

    except Exception as e:
        print(f"\n[Exceção] Erro ao processar {nome_dep}: {e}")
    
    # Delay para não sobrecarregar o servidor
    time.sleep(0.5)

print("\n\nColeta Finalizada!")

# ---------------------------------------------------------
# 3. Cria o DataFrame Final
# ---------------------------------------------------------
df_viagens = pd.DataFrame(lista_viagens)

# Mostra o resultado
print(f"Total de viagens encontradas: {len(df_viagens)}")
df_viagens.head()

In [40]:
df_viagens.to_csv('Viagens Oficiais Dep.csv', index=False)

## Auxilio moradia

In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# ---------------------------------------------------------
# 1. Configuração
# ---------------------------------------------------------
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

lista_auxilios = []

print(f"Iniciando coleta de Auxílio-Moradia (2025) para {len(df)} deputados...\n")

# ---------------------------------------------------------
# 2. Loop Principal
# ---------------------------------------------------------
for index, row in df.iterrows():
    id_dep = row['id']
    nome_dep = row['nome']
    
    # URL específica para auxílio-moradia
    url = f"https://www.camara.leg.br/deputados/{id_dep}/auxilio-moradia?ano=2025"
    
    print(f"Verificando: {nome_dep} (ID: {id_dep})...", end='\r')
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Procura a div específica que envolve a tabela de auxílio
            # No arquivo html auxilio.txt, a classe é "tabela__detalhamento"
            div_tabela = soup.find('div', class_='tabela__detalhamento')
            
            if div_tabela:
                tabela = div_tabela.find('table')
                
                if tabela:
                    tbody = tabela.find('tbody')
                    linhas = tbody.find_all('tr')
                    
                    for tr in linhas:
                        cols = tr.find_all('td')
                        
                        # A tabela tem apenas 2 colunas: Mês e Valor
                        if len(cols) >= 2:
                            mes = cols[0].get_text(strip=True)
                            valor = cols[1].get_text(strip=True)
                            
                            lista_auxilios.append({
                                'ID Deputado': id_dep,
                                'Nome Deputado': nome_dep,
                                'Mês': mes,
                                'Valor Auxílio (R$)': valor,
                                'Ano': 2025
                            })
            else:
                # É comum não ter a tabela se o deputado ocupa imóvel funcional
                pass
                
        else:
            print(f"\n[Erro] Status {response.status_code} para {nome_dep}")

    except Exception as e:
        print(f"\n[Exceção] Erro ao processar {nome_dep}: {e}")
    
    time.sleep(0.5)

print("\n\nColeta Finalizada!")

# ---------------------------------------------------------
# 3. Cria o DataFrame Final
# ---------------------------------------------------------
df_auxilio = pd.DataFrame(lista_auxilios)

print(f"Total de registros encontrados: {len(df_auxilio)}")
df_auxilio.head()

Iniciando coleta de Auxílio-Moradia (2025) para 17 deputados...

Verificando: Carlos Chiodini (ID: 204361)...32)...

Coleta Finalizada!
Total de registros encontrados: 22


,ID Deputado,Nome Deputado,Mês,Valor Auxílio (R$),Ano
0,178966,Geovania de Sá,01,"4.253,00",2025
1,178966,Geovania de Sá,02,"4.253,00",2025
2,178966,Geovania de Sá,03,"4.253,00",2025
3,178966,Geovania de Sá,04,"4.253,00",2025
4,178966,Geovania de Sá,05,"4.253,00",2025


In [44]:
df_auxilio.to_csv('Auxilio moradia dep.csv', index=False)

## Consolidado Gastos pessoal 2025

In [3]:
df_dep

,Unnamed: 0,id,nome_civil,nome,url,partido
0,0,220556,ANA PAULA DE SOUZA LIMA,Ana Paula Lima,https://www.camara.leg.br/internet/deputado/ba...,PT
1,1,204369,CAROLINE RODRIGUES DE TONI,Caroline de Toni,https://www.camara.leg.br/internet/deputado/ba...,PL
2,2,98148,VALDIR VITAL COBALCHINI,Cobalchini,https://www.camara.leg.br/internet/deputado/ba...,MDB
3,3,204367,DANIEL COSTA DE FREITAS,Daniel Freitas,https://www.camara.leg.br/internet/deputado/ba...,PL
4,4,220557,DANIELA CRISTINA REINEHR,Daniela Reinehr,https://www.camara.leg.br/internet/deputado/ba...,PL
5,5,204368,FÁBIO LUIZ SCHIOCHET FILHO,Fabio Schiochet,https://www.camara.leg.br/internet/deputado/ba...,UNIÃO
6,6,178966,Geovânia de Sá,Geovania de Sá,https://www.camara.leg.br/internet/deputado/ba...,PSDB
7,7,204365,GILSON MARQUES VIEIRA,Gilson Marques,https://www.camara.leg.br/internet/deputado/ba...,NOVO
8,8,98615,ISMAEL DOS SANTOS,Ismael,https://www.camara.leg.br/internet/deputado/ba...,PSD
9,9,214694,JORGE GOETTEN DE LIMA,Jorge Goetten,https://www.camara.leg.br/internet/deputado/ba...,REPUBLICANOS


In [14]:
import requests
from bs4 import BeautifulSoup
from io import StringIO

dfs = []
for index, linha in df_dep.iterrows():
    nome = linha['nome_civil']
    id_ = linha['id']
    url = f'https://www.camara.leg.br/deputados/{id_}/verba-gabinete?ano=2025'

    print(f'Acessando Gastos do deputado {nome} \n url: {url}', end='\r')
    
    response = requests.get(url)
    if response.status_code == 200:
        html = StringIO(response.text)
        df = pd.read_html(html)[0]
        df['Deputado'] = nome
        dfs.append(df)
        time.sleep(2)
    else:
        print(f'{nome} Mal sucedido!')
        
df_final_consolidado = pd.concat(dfs)

Acessando Gastos do deputado ANA PAULA DE SOUZA LIMA 
Acessando Gastos do deputado CAROLINE RODRIGUES DE TONI abinete?ano=2025
Acessando Gastos do deputado VALDIR VITAL COBALCHINI a-gabinete?ano=2025
Acessando Gastos do deputado DANIEL COSTA DE FREITAS -gabinete?ano=2025
Acessando Gastos do deputado DANIELA CRISTINA REINEHR -gabinete?ano=2025
Acessando Gastos do deputado FÁBIO LUIZ SCHIOCHET FILHO abinete?ano=2025
Acessando Gastos do deputado Geovânia de Sá 4368/verba-gabinete?ano=2025
Acessando Gastos do deputado GILSON MARQUES VIEIRA rba-gabinete?ano=2025
Acessando Gastos do deputado ISMAEL DOS SANTOS 5/verba-gabinete?ano=2025
Acessando Gastos do deputado JORGE GOETTEN DE LIMA ba-gabinete?ano=2025
Acessando Gastos do deputado JULIA PEDROSO ZANATTA rba-gabinete?ano=2025
Acessando Gastos do deputado LUIZ FERNANDO CARDOSO rba-gabinete?ano=2025
Acessando Gastos do deputado PEDRO FRANCISCO UCZAI rba-gabinete?ano=2025
Acessando Gastos do deputado RAFAEL PEZENTI 0604/verba-gabinete?ano=2025

In [ ]:
len(df_final_consolidado.Deputado.unique())

18

In [18]:
df_final_consolidado.to_csv('Verba Gabinete 2025.csv')